In [1]:
from time import time

__import__("warnings").filterwarnings('ignore')

global_time = time()
global_time_stop = False

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathon-2025/aggregate_data_files_log10.pkl
/kaggle/input/hackathon-2025/preprocess_files_log10.pkl
/kaggle/input/hackathon-2025/aggregate_data_files_no_log.pkl
/kaggle/input/hackathon-2025/preprocess_files_no_log.pkl


In [3]:
!pip install tqdm_joblib -q
!pip install transformers==4.47.0 -q
!pip install category-encoders==2.8.0 -q
!pip install scikit-learn==1.6.0 -q
!pip install catboost==1.2.7 -q
!pip install featuretools==1.31.0 -q
!pip install matplotlib==3.8.3 -q
!pip install optuna==4.1.0 -q
!pip install optuna-integration==4.1.0 -q
!pip install LightAutoML==0.4.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 72.2 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 72.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 75.7 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.6/399.6 kB 6.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.0/216.0 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 71.9 MB/s eta 0:00:00:00:010:01


In [4]:
# Скачиваем дополнительные модули из репо
!wget -q https://raw.githubusercontent.com/saspav/DFC-2025/main/df_addons.py > /dev/null 2>&1
!wget -q https://raw.githubusercontent.com/saspav/DFC-2025/main/print_time.py > /dev/null 2>&1
!wget -q https://raw.githubusercontent.com/saspav/DFC-2025/main/set_all_seeds.py > /dev/null 2>&1
!wget -q https://raw.githubusercontent.com/saspav/DFC-2025/main/data_process_sml.py > /dev/null 2>&1

In [5]:
import numpy as np
import pandas as pd
import copy
import joblib

from pathlib import Path

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
# from lightautoml.automl.presets.whitebox_presets import WhiteBoxPreset
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from sklearn.metrics import mean_squared_log_error, mean_absolute_error

from data_process_sml import (RANDOM_SEED, PREDICTIONS_DIR, MODEL_PATH, get_max_num,
                              add_info_to_log, DataTransform)

from print_time import print_time, print_msg

from set_all_seeds import set_all_seeds

__import__("warnings").filterwarnings('ignore')

set_all_seeds(seed=RANDOM_SEED)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

DATASET_PATH = Path('/kaggle/input/hackathon-2025')

WORK_PATH = Path('.')
MODELS_LOGS = WORK_PATH.joinpath('scores.logs')
MODELS_LOGS_REG = WORK_PATH.joinpath('scores_reg.logs')

In [6]:
DATASET_PATH, PREDICTIONS_DIR, MODEL_PATH

(PosixPath('/kaggle/input/hackathon-2025'),
 PosixPath('predictions'),
 PosixPath('models'))

In [7]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [8]:
def acc_score(y_true, y_pred, **kwargs):
    return accuracy_score(y_true, (y_pred > 0.5).astype(int), **kwargs)


def f1_metric(y_true, y_pred, **kwargs):
    return f1_score(y_true, (y_pred > 0.5).astype(int), **kwargs)
    

start_time = print_msg('Обучение TabularAutoML ...')

max_num_bin = get_max_num(log_file=MODELS_LOGS) + 1
max_num_reg = get_max_num(log_file=MODELS_LOGS_REG) + 1

sub_pref = 'kla_'

numeric_columns = []

binary_targets = ['Некачественное ГДИС', 'Влияние ствола скважины', 'Радиальный режим',
                  'Линейный режим', 'Билинейный режим', 'Сферический режим',
                  'Граница постоянного давления', 'Граница непроницаемый разлом']

numeric_targets = ['Влияние ствола скважины_details', 'Радиальный режим_details',
                   'Линейный режим_details', 'Билинейный режим_details',
                   'Сферический режим_details', 'Граница постоянного давления_details',
                   'Граница непроницаемый разлом_details']

# binary_targets = [clean_column_name(col) for col in binary_targets]

cat_columns = []

# Чтение и предобработка данных
data_cls = DataTransform(use_catboost=True,
                         category_columns=cat_columns,
                         drop_first=False,
                         # numeric_columns=numeric_columns, scaler=StandardScaler,
                         )

# train_df, test_df = data_cls.make_agg_data(use_featuretools=False,
#                                            file_with_target_class=None)

# Только ручные признаки
preprocess_files = DATASET_PATH.joinpath('preprocess_files_log10.pkl')
with open(preprocess_files, 'rb') as in_file:
    train_df, test_df, *_ = joblib.load(in_file)

# # С признаками TSFRESH
# aggregate_path_file = DATASET_PATH.joinpath('aggregate_data_files_log10.pkl')
# with open(aggregate_path_file, 'rb') as in_file:
#     train_df, test_df = joblib.load(in_file)

# train_df = train_df.sample(n=300, random_state=RANDOM_SEED)

# # Добавление группировок от таргет-енкодинга
# train_df = data_cls.fit_transform(train_df)
# test_df = data_cls.transform(test_df)

features2drop = ['hq', 'labels']

exclude_columns = ['count_rows',
                   ]

# exclude_columns = [clean_column_name(col) for col in exclude_columns]

exclude_columns.extend(data_cls.exclude_columns)

model_columns = test_df.columns.to_list()

# Добавим в категориальные признаки те, что были посчитаны как мода
cat_columns.extend([col for col in model_columns if col.upper().startswith('MODE_')])

model_columns = [col for col in model_columns if col not in exclude_columns]
cat_columns = [col for col in cat_columns if col in model_columns]

exclude_columns = features2drop + exclude_columns

print('Обучаюсь на колонках:', model_columns)
print('Категорийные колонки:', cat_columns)
print('Исключенные колонки:', exclude_columns)

print(f'Размер train_df = {train_df.shape}, test = {test_df.shape}')

train = train_df[model_columns].drop(columns=features2drop, errors='ignore')
target = train_df[binary_targets + numeric_targets]
test_df = test_df[model_columns].copy()

print('train.shape', train.shape, 'пропусков:', train.isna().sum().sum())
print('test.shape', test_df.drop(columns=features2drop, errors='ignore').shape,
      'пропусков:', test_df.isna().sum().sum())

# set initial runtime rate guess for first level models
_time_scores = {
    "lgb": 1,
    "lgb_tuned": 3,
    "xgb": 1,
    "xgb_tuned": 3,
    "linear_l2": 0.7,
    "cb": 2,
    "cb_tuned": 6,
    "rf": 5,
    "rf_tuned": 10,
    "nn": 10,
    "nn_tuned": 20,
}

N_THREADS = 4  # threads cnt for lgbm and linear models
G_MEMORY = 24  # 
N_FOLDS = 5  # folds cnt for AutoML
TEST_SIZE = 0.2  # Test size for metric check
TIMEOUT = 60 * 45  # Time in seconds for automl run

Обучение TabularAutoML ...
Обучаюсь на колонках: ['dP_mean', 'dP_median', 'dP_std', 'Pder_mean', 'Pder_median', 'Pder_std', 'wb_present', 'wb_b', 'ra_present', 'ra_b', 'li_present', 'li_b', 'bi_present', 'bi_b', 'sp_present', 'sp_b', 'pc_time', 'im_time']
Категорийные колонки: []
Исключенные колонки: ['hq', 'labels', 'count_rows']
Размер train_df = (45141, 38), test = (500, 19)
train.shape (45141, 18) пропусков: 0
test.shape (500, 18) пропусков: 0


In [ ]:
full_submit = pd.DataFrame(index=test_df.index)
models_scores = []
models_to_save = []

for idx_target, one_target in enumerate(binary_targets + numeric_targets):

    train_with_target = train.join(target[one_target], how="left")

    binary_task = True
    if idx_target > 7:
        binary_task = False

    if binary_task:
        # Available metrics for binary task:
        # ‘auc’ - (uses by default) ROC-AUC score.
        # ‘accuracy’ - Accuracy score (uses argmax prediction).
        # ‘logloss’ - Standard logistic loss.
        task = Task('binary', metric='auc')
    else:
        # Avaliable losses & metrics for regression task:
        # ‘mse’ - (uses by default) Mean Squared Error.
        # ‘mae’ - Mean Absolute Error.
        # ‘mape’ - Mean Absolute Percentage Error.
        # ‘rmsle’ - Root Mean Squared Log Error.
        task = Task('reg', loss='mse', metric='mse')

    roles = {'target': one_target,
             }

    RD = ReportDeco(output_path='tabularAutoML_model_report')

    # Настройка автоматической модели
    automl = TabularAutoML(task=task,
                           timeout=TIMEOUT,
                           cpu_limit=N_THREADS,
                           memory_limit=G_MEMORY,
                           general_params={
                               # 'use_algos': [[
                               #     'rf',
                               #     'lgb',
                               #     'lgb_tuned',
                               #     'cb_tuned',
                               #     # 'linear_l2',
                               #     'xgb_tuned',
                               # ],
                               #     [
                               #         'cb_tuned',
                               #         # 'rf',
                               #         # 'linear_l2',
                               #     ],
                               # ],

                               'use_algos': 'auto',

                               # 'use_algos': [[
                               #     'rf',
                               #     'lgb',
                               #     'cb', ],
                               # ],

                               'tuning_params': {
                                   # Время для подбора гиперпараметров (в секундах)
                                   'max_tuning_time': 60 * 12,
                                   # Максимальное количество итераций подбора
                                   'max_tuning_iter': 88,
                               },                               

                               # 'custom_params': custom_params,
                           },
                           reader_params={'n_jobs': N_THREADS,
                                          'cv': N_FOLDS,
                                          'random_state': RANDOM_SEED},
                           )

    # automl = RD(automl)

    oof_pred = automl.fit_predict(train_with_target, roles=roles, verbose=1)
    valid_proba = oof_pred.data[:, 0]

    print('oof_pred:\n{}\nShape = {}'.format(oof_pred[:7], oof_pred.shape))

    test_pred = automl.predict(test_df)
    predict_test = test_pred.data[:, 0]

    models_to_save.append(automl)

    print('Check scores...')

    y_valid = train_with_target[one_target].values

    if binary_task:
        predict_valid = (valid_proba >= 0.5).astype(int)
        predict_test = (predict_test >= 0.5).astype(int)
        model_score = acc_score(y_valid, valid_proba)
    else:
        # Заменяем значения меньше -33 на 0, т.к. пропуски были заполнены -99
        y_valid[y_valid < -33] = 0
        valid_proba[valid_proba < -33] = 0
        model_score = mean_squared_error(y_valid, valid_proba)

    print('OOF score: {}'.format(model_score))

    submission = test_df[test_df.columns.to_list()[:2]].copy()
    submission[one_target] = predict_test
    full_submit[one_target] = predict_test

    # Сохранение предсказаний в файл
    max_num = max_num_bin
    post_fix = ''
    if not binary_task:
        post_fix = '_reg'
        max_num = max_num_reg
    submit_csv = f'{sub_pref}submit_{max_num:03}_target{idx_target}{post_fix}.csv'

    submission[[one_target]].to_csv(PREDICTIONS_DIR.joinpath(submit_csv))

    if idx_target == 7:
        # Сохранение предсказаний в файл
        submit_csv = f'{sub_pref}submit_{max_num:03}.csv'
        full_submit.to_csv(PREDICTIONS_DIR.joinpath(submit_csv))

    print(automl.create_model_str_desc())

    info_cols = (model_columns, exclude_columns, cat_columns)

    save_time = print_msg('Сохранение модели ...')
    save_vodel_file = MODEL_PATH.joinpath(f'automl_{max_num:03}_{idx_target:02}.pkl')
    with open(save_vodel_file, 'wb') as file:
        joblib.dump((automl, binary_targets, numeric_targets, info_cols), file,
                    compress=7)
    print_time(save_time)

    comment = {}
    comment.update({'SEED': RANDOM_SEED,
                    'target': one_target,
                    })

    t_score = f1_macro = f1_micro = f1_wght = 0
    if binary_task:
        # Для многоклассового ROC AUC, нужно указать multi_class
        auc_macro = roc_auc_score(y_valid, valid_proba, average='macro')
        auc_micro = roc_auc_score(y_valid, valid_proba, average='micro')
        auc_wght = roc_auc_score(y_valid, valid_proba, average='weighted')
        try:
            f1_macro = f1_score(y_valid, predict_valid, average='macro')
            f1_micro = f1_score(y_valid, predict_valid, average='micro')
            f1_wght = f1_score(y_valid, predict_valid, average='weighted')
        except:
            pass
    else:
        auc_macro = mean_squared_error(y_valid, predict_valid) ** 0.5
        # Mean Absolute Error: среднее абсолютное отклонение предсказанных значений от фактических
        auc_micro = mean_absolute_error(y_valid, predict_valid)
        # Mean Squared Error: средний квадрат отклонений предсказаний от фактических значений
        auc_wght = mean_squared_error(y_valid, predict_valid)
        # R² Score показывает, какую долю вариации зависимой переменной объясняет модель. Значение
        # R² варьируется от 0 до 1 (или может быть отрицательным, если модель плоха).
        # Значение 1 говорит о том, что модель идеально подходит к данным.
        f1_macro = r2_score(y_valid, predict_valid)
        # Explained Variance Score: метрика показывает долю вариации целевой переменной, которую
        # может объяснить модель. Значение равно 1, если модель идеально объясняет данные, и
        # 0, если модель не способна предсказать значение лучше, чем просто использовать среднее
        # значение целевой переменной.
        f1_wght = explained_variance_score(y_valid, predict_valid)
        # Mean Squared Logarithmic Error: измеряет среднюю разницу между логарифмами предсказанных
        # и фактических значений. Это полезно в случаях, когда хотим уменьшить влияние больших
        # ошибок, особенно при работе с экспоненциально растущими данными.
        # Заменяем значения меньше нуля на 0
        try:
            y_valid[y_valid < 0] = 0
            predict_valid[predict_valid < 0] = 0
            f1_micro = mean_squared_log_error(y_valid, predict_valid)
        except:
            pass

    valid_scores = (model_score, auc_macro, auc_micro, auc_wght,
                    f1_macro, f1_micro, f1_wght, t_score)

    models_scores.append(valid_scores)

    if binary_task:
        add_info_to_log(f'{sub_pref}{idx_target}_', max_num_bin, 0, None, valid_scores,
                        info_cols, comment, log_file=MODELS_LOGS)
    else:
        add_info_to_log(f'{sub_pref}{idx_target}_', max_num_reg, 0, None, valid_scores,
                        info_cols, comment, log_file=MODELS_LOGS_REG)

# Постпроцессинг: зануляем значения, если binary_targets = 0
for target, binary_col in zip(numeric_targets, binary_targets[1:]):
    full_submit.loc[full_submit[binary_col] == 0, target] = np.nan
# Сохранение предсказаний в файл
submit_csv = f'{sub_pref}submit_{max_num_reg:03}_reg.csv'
full_submit.to_csv(PREDICTIONS_DIR.joinpath(submit_csv))

save_time = print_msg('Сохранение моделей ...')
with open(MODEL_PATH.joinpath(f'automl_{max_num_reg:03}_all.pkl'), 'wb') as file:
    joblib.dump((models_to_save, binary_targets, numeric_targets, info_cols), file,
                compress=7)
print_time(save_time)

comment = {}
comment.update({'SEED': RANDOM_SEED,
                'target': f'Кол-во: {len(binary_targets)}',
                })

valid_scores = [np.mean(arg) for arg in zip(*models_scores[:8])]
add_info_to_log(f'{sub_pref}{idx_target}_', max_num_bin, 0, None, valid_scores,
                info_cols, comment, log_file=MODELS_LOGS)

valid_scores = [np.mean(arg) for arg in zip(*models_scores[8:])]
add_info_to_log(f'{sub_pref}{idx_target}_', max_num_reg, 0, None, valid_scores,
                info_cols, comment, log_file=MODELS_LOGS_REG)

print_time(start_time)


[17:52:47] Stdout logging level is INFO.
[17:52:47] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[17:52:47] Task: binary

[17:52:47] Start automl preset with listed constraints:
[17:52:47] - time: 2700.00 seconds
[17:52:47] - CPU: 4 cores
[17:52:47] - memory: 24 GB

[17:52:47] Train data shape: (45141, 19)

[17:52:53] Layer 1 train process start. Time left 2693.57 secs
[17:52:55] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[17:52:58] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.656767177914291
[17:52:58] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[17:52:58] Time left 2689.25 secs

[17:52:59] Selector_LightGBM fitting and predicting completed
[17:53:01] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[17:53:16] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.8542541608000961
[17:53:16] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[17:53:16] Start hyperparameters optimization for Lv

Optimization Progress:  94%|█████████▍| 95/101 [05:01<00:19,  3.17s/it, best_trial=12, best_value=0.857]

[17:58:17] Hyperparameters optimization for Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM completed
[17:58:17] Start fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM ...


[17:58:27] Fitting Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM finished. score = 0.8518587092053268
[17:58:27] Lvl_0_Pipe_1_Mod_1_Tuned_LightGBM fitting and predicting completed
[17:58:27] Start fitting Lvl_0_Pipe_1_Mod_2_CatBoost ...
[17:59:16] Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = 0.8499296233157536
[17:59:16] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed
[17:59:16] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 300.00 secs


Optimization Progress:   0%|          | 0/101 [00:00<?, ?it/s]

In [ ]:
from IPython.display import display, FileLink
from zipfile import ZipFile, ZIP_DEFLATED as ZD
from glob import glob

files = glob(str(MODEL_PATH) + '/*.*') + glob(str(PREDICTIONS_DIR) + '/*.*')
files += glob('tabularAutoML_model_report/*.*') 
files += glob('*.logs')
zip_filename = f'model_lightautoml_clf_reg.zip'
with ZipFile(zip_filename, 'w',  compression=ZD, compresslevel=9) as zip_file:
    for filename in files:
        print(filename)
        zip_file.write(filename)
FileLink(zip_filename)